In [1]:
import pandas as pd
import numpy as np
import pdfplumber

import time
import os
import sys
from pathlib import Path

import re


import PyPDF2



In [11]:
def get_file_name(file_path):
    
    '''get files under certain path

    Args:
        file path,string

    Returns:
        A dictionary for stroing the file name and it's path

    Raise;
        ValueError,file path name

    '''
    file_dir ={}
    for dirpath, dirnames, files in os.walk(file_path, topdown=False): ## work down certain dir,has 3 returns
#         print(f'Found directory: {dirpath}')
        for file_name in files:
            if file_name.endswith('.pdf'):
                file_dir[file_name] = os.path.join(dirpath,file_name)
#             print(os.path.join(dirpath,file_name))
            
    return file_dir

In [12]:
file_dir = get_file_name('./well_logs')

In [15]:
def get_survey_path(file_dir):
    
    '''get file contain 'survey'

    Args:
        file path,string

    Returns:
        A dictionary for stroing the file name and it's path

    Raise;
     
    '''
    survey_path ={}
    pattern = r'.*[Ss]urvey' ##pattern
    for file,path in file_dir.items():
        find_file = re.search(pattern,file) ## find the file with such pattern
        if find_file:
            file_name = find_file.group(0) ##save the name for matching the pattern
            survey_path[file_name] = path ## save file name to dictionary
            
    return survey_path
            

In [9]:

def decrpyt_pdf(input_file,out_put_file,searching_item):
    '''decrpted file with qpdf from command line

    Args:
        input_file ,oup_put_file path and searching_item,string

    Returns:
        oup_put_file path,string

    Raise;
        file not found error
     
    '''
     
    PASSWORD = '123' ## set a random guess password
    try:
        with open(input_file, mode='rb') as f:  ## open the pdf file in rb mode 
            reader = PyPDF2.PdfFileReader(f) # create a pdffileread object
            if reader.isEncrypted: # check if the file is encryped or not
                try:
                    reader.decrypt(PASSWORD) ## give a random guess
                ## catch the wroing guess and decrypt the input file
                except NotImplementedError: 
                    command= f"qpdf  --decrypt {input_file} {out_put_file};" ## concate the command 
                    os.system(command) ## run the comman inside the cmd
                    ## open the decrypted file and perform function
                    with open(FILE_OUT_PATH, mode='rb') as fp:
                        reader = PyPDF2.PdfFileReader(fp) ## generate reader object
                        counters = read_page_numbers(reader,searching_item) ## perform function 
            else:
                ## if not encrypted, then perform searching function
                counters = read_page_numbers(reader,searching_item)
    ## catch file not found error
    except OSError as e:
        print(e)

In [21]:
survey_path = get_survey_path(file_dir)

In [32]:
def get_survey_content(survey_path):

    '''get pdf content from the survey_path

    Args:
        survey_path, dict

    Returns:
        tables generator

    Raise;
        file not found error
    '''
    file_table = {}
    for file,path in survey_path.items():
        tables = [] ## save each page table to tables container-one file has one tables
        pdf = pdfplumber.open(path)
        print('preprocess file of',path)
        for page in pdf.pages:
        #     print(page.extract_text())
            for pdf_table in page.extract_tables():
                table = [] ## each page saved to one table
                cells = []
                for row in pdf_table:
                    if not any(row):
                        # 如果一行全为空，则视为一条记录结束
                        if any(cells):
                            table.append(cells)
                            cells = []
                    elif all(row):
                        # 如果一行全不为空，则本条为新行，上一条结束
                        if any(cells):
                            table.append(cells)
                            cells = []
                        table.append(row)
                    else:
                        if len(cells) == 0:
                            cells = row
                        else:
                            for i in range(len(row)):
                                if row[i] is not None:
                                    cells[i] = row[i] if cells[i] is None else cells[i] + row[i]
                tables.append(table) ## append each page to one file container
                
                for row in table:
                    row = [re.sub('\s', '}', cell) if cell is not None else None for cell in row]
#                 print('---------- dash line ----------') ## end of preprocess the file pages
        file_table[path] = tables
                


#                 pdf.close()|

        yield file_table


In [ ]:
preprocessed_file ={}
for file_pages in get_survey_content(survey_path):
    
     ##for each file
    for file,tables in file_pages.items() : ##iterate through
#         print(file,tables)
        file_tables = []
        for table in tables:
            table_rows = []
            if len(table) == 8 or len(table) == 9:
                for i,row in enumerate(table):
    #                 print(f'this is {i} row from one table',row,type(row))
                    row_np = np.array(row)
                    row_np_squeeze = row_np.squeeze()
                    table_rows.append(row_np_squeeze) ## append rows to table
#                     print(row_np_squeeze.shape)
                    print(f'this is {i} row from one table','np squeeze file',row_np_squeeze,end='')
            file_tables.append(table_rows) ## append table to tables
        print('-------end of file-----------') ## end of preprocess the file
    preprocessed_file[file] = file_tables ## correlate file with tables
        


preprocess file of ./well_logs\well_logs\01\trackingNo_203729\05535133 Gyro Survey001.pdf
-------end of file-----------
preprocess file of ./well_logs\well_logs\01\trackingNo_203729\42-055-35133 Surveys.pdf
-------end of file-----------
-------end of file-----------
preprocess file of ./well_logs\well_logs\01\trackingNo_205798\Korn A 3H Surveys.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
preprocess file of ./well_logs\well_logs\01\trackingNo_212488\MG LILLIE 1616 GU B 7H-Final Gyro Survey 0-4231.36.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
preprocess file of ./well_logs\well_logs\08\trackingNo_222775\Survey Report-Garrett-Snell Unit A 25-36 No. 1SH(Final)_.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
preprocess file of ./well_logs\well_logs\7C\track

preprocess file of ./well_logs\well_logs\01\trackingNo_216237\Newman 2H Surveys.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

...

preprocess file of ./well_logs\well_logs\01\trackingNo_217446\Faith-Yana GG Unit O 1H ST Surveys.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

preprocess file of ./well_logs\well_logs\01\trackingNo_217759\Piloncillo A 45HB Surveys.pdf
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
-------end of file-----------
this is 0 row from one table np squeeze file CLOSURE CLOSURE
MD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR
ft deg deg ft ft ft ft °/100ft ft degthis is 1 row from one table np squeeze file 591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90
710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20
805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76
899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31
994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93this is 2 row from one table np squeeze file 1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00
1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62
1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79
1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83
1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 2

In [167]:

for tables in get_survef_content(survey_path):
    for table in tables:
        table_np = np.array(table)
        table_np_squ = table_np.squeeze()
    #     print(table_np_squ)
        for tab in table_np_squ:
#             print(tab)
            if tab.any():
                split_line = tab.split('}')
                print(split_line,'this is a line')
                break

preprocess file of ./well_logs\well_logs\01\trackingNo_203729\05535133 Gyro Survey001.pdf
preprocess file of ./well_logs\well_logs\01\trackingNo_203729\42-055-35133 Surveys.pdf
preprocess file of ./well_logs\well_logs\01\trackingNo_205798\Korn A 3H Surveys.pdf
preprocess file of ./well_logs\well_logs\01\trackingNo_212488\MG LILLIE 1616 GU B 7H-Final Gyro Survey 0-4231.36.pdf
preprocess file of ./well_logs\well_logs\08\trackingNo_222775\Survey Report-Garrett-Snell Unit A 25-36 No. 1SH(Final)_.pdf
['*o8pany:}Surge}Energy}LLC})oca3}*o"or-inate}Re9erence:},ell}No\'}(SH}$ro7ect:}Howard}County,}TX}(NAD27)}%:D}Re9erence:},ell}-}2#./\'//u01t}(GL2}2##34}5}672}224}(8att"#9))}Site:}Garrett-Snell}}n}t}A}2!-"#}MD}Re9erence:},ell}-}2#./\'//u01t}(GL2}2##34}5}672}224}(8att"#9))}We33:}No\'}(SH}0orth}Re9erence:}Gr}d}We33;ore:})H}Survey}*a3cu3ation}Metho-:}*}n}+u+}Cur$ature}Design:}Gyro:*,D}!3#4-(72//4}8T7}(727"4}Data;ase:},ell8lanner(']
['$ro7ect}Howard}County,}TX}(NAD27)']
['Map}Syste8:}}S}State}8lane}

TypeError: cannot perform reduce with flexible type

In [125]:
tables =[[], 
         [['CLOSURE CLOSURE\nMD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR\nft deg deg ft ft ft ft °/100ft ft deg'],
          ['591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90\n710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20\n805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76\n899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31\n994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93'], ['1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00\n1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62\n1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79\n1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83\n1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 224.80'], ['1563.00 6.79 214.40 1561.06 33.83 -33.17 -31.66 1.43 45.85 223.67\n1657.00 6.69 204.60 1654.42 43.51 -42.73 -37.08 1.23 56.58 220.95\n1752.00 7.13 206.77 1748.73 53.91 -53.02 -42.04 0.54 67.67 218.41\n1847.00 6.56 209.02 1843.05 64.02 -63.03 -47.32 0.66 78.82 216.90\n1942.00 6.71 210.06 1937.41 73.69 -72.58 -52.74 0.21 89.72 216.00'], ['2036.00 6.65 202.24 2030.78 83.58 -82.38 -57.55 0.97 100.49 214.94\n2131.00 7.07 193.30 2125.10 94.44 -93.16 -60.98 1.21 111.34 213.21\n2226.00 6.74 195.21 2219.41 105.57 -104.23 -63.79 0.42 122.20 211.47\n2321.00 7.59 203.93 2313.67 116.76 -115.35 -67.80 1.45 133.80 210.45\n2415.00 7.56 206.42 2406.85 128.08 -126.56 -73.07 0.35 146.13 210.00'], ['2510.00 7.37 206.03 2501.04 139.27 -137.63 -78.52 0.21 158.46 209.71\n2604.00 6.34 212.93 2594.37 149.17 -147.41 -83.99 1.40 169.66 209.67\n2699.00 6.47 211.17 2688.78 158.26 -156.39 -89.61 0.24 180.25 209.81\n2794.00 6.63 213.42 2783.16 167.54 -165.55 -95.40 0.32 191.07 209.95\n2888.00 6.74 216.60 2876.52 176.63 -174.50 -101.68 0.41 201.96 210.23'], ['2983.00 6.30 216.50 2970.91 185.43 -183.17 -108.10 0.46 212.69 210.55\n3078.00 6.40 216.93 3065.32 193.98 -191.59 -114.38 0.12 223.14 210.84\n3172.00 6.13 221.78 3158.76 202.05 -199.52 -120.87 0.63 233.28 211.21\n3267.00 6.30 220.91 3253.21 209.91 -207.24 -127.66 0.21 243.40 211.63\n3361.00 6.04 221.08 3346.66 217.67 -214.86 -134.29 0.28 253.37 212.01'], ['3456.00 5.26 219.29 3441.20 224.94 -222.00 -140.33 0.84 262.63 212.30\n3551.00 5.15 210.96 3535.81 232.07 -229.02 -145.28 0.80 271.22 212.39']]]

In [143]:
for table in tables:
    table_np = np.array(table)
    table_np_squ = table_np.squeeze()
#     print(table_np_squ)
    for tab in table_np_squ:
        if tab:
            split_line = tab.split(' ')
            print(split_line)
            

['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['591.00', '0.50', '208.12', '590.99', '2.09', '-2.07', '-1.05', 'Invalid', '2.32', '206.90\n710.00', '0.43', '222.59', '709.99', '2.89', '-2.86', '-1.60', '0.11', '3.27', '209.20\n805.00', '0.81', '205.75', '804.98', '3.77', '-3.72', '-2.13', '0.44', '4.29', '209.76\n899.00', '0.40', '244.23', '898.98', '4.52', '-4.46', '-2.71', '0.59', '5.22', '211.31\n994.00', '0.75', '240.95', '993.97', '4.99', '-4.91', '-3.56', '0.36', '6.06', '215.93']
['1089.00', '0.37', '213.66', '1088.97', '5.56', '-5.47', '-4.27', '0.48', '6.94', '218.00\n1183.00', '2.55', '234.89', '1182.93', '7.05', '-6.92', '-6.15', '2.35', '9.26', '221.62\n1278.00', '4.71', '226.86', '1277.73', '11.03', '-10.81', '-10.73', '2.33', '15.23', '224.79\n1373.00', '6.15', '223.40', '1372.31', '17.53', '-17.17', '-17.07', '1.56', '24.21', '224.83\n1468.00', '6.66', '225.

In [127]:
np_tab.shape

(1, 1, 0)